V tomto notebooku si ukážeme, jak jednoduše načíst marcový dokument pomocí knihovny pymarc, jak marcový záznam vypadá, jak data z marcového dokumentu vytáhnout a uložit do csv tabulky. Nakonec z dat zjistíme nejčetnějšího autora za každý rok.   


### 0. Příprava 
Jako první si musíme nainstalovat knihovny, se kterými budeme pracovat. Knihovny jsou balíčky funkcí, které nejsou součástí základu jazyka python. <br>
Knihovny nainstalujeme pomocí příkazu `%pip install <jmeno_knihovny>` . Pak je do našeho notebooku přidáme pomocí příkazu `import <jmeno_knihovny> (as alias)`. K funkcím knihovny se pak přistupuje `jmeno_knihovny.jmeno_funkce` <br> 
Pokud z knihovny chceme využít pouze jednu funkci, přidáme ji pomocí `from <jmeno_knihovny> import <jmeno_funkce>`

In [46]:
# Prikaz ktery naistaluje knihovny
%pip install pandas 
%pip install pymarc openpyxl
%pip install openpyxl

# Prikaz ktery knihovny prida 
from collections import Counter
import pandas as pd
from pymarc import MARCReader


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### 1. Načtení MARC souboru

Data si stáhneme a podíváme se, co je uvnitř. Jsou uložena v knihovním formátu MARC s koncovnkou .mrc. Pro práci budeme používat knihovnu pymarc, která dokáže marcový formát jednodušše načíst. <br>   
Nejprve si vypíšeme, jak takový záznam vypadá.  

In [47]:
# Cesta k marcovemu dokumentu
database = 'data/ucla/ucla_cle.mrc'

# i-ty zaznam, ktery chceme vypsat
ith = 5

# Otevreni souboru
with open(database, 'rb') as data:
    # Nacteni marcu
    reader = MARCReader(data)

    # Iterace skrz nacteny soubor
    for i,record in enumerate(reader):        
        
        # Pokud je i nas zaznam, vypiseme ho 
        if i <= ith:
            # Vypsani marcovych zaznamu
            print(record)

            # Chceme vypsat pouze jeden zaznam, tak po prvnim vypsani zavolame funkci break
            break


=LDR  00875nab a22003134a 4500
=001  001457852
=003  CZ\PrUCL
=005  20220321104606.0
=008  090923e197808\\xr\\\\\\\\\\\\|||\||cze\d
=035  \\$a(ISIS-B70-MFN)78971
=035  \\$a(ISIS-B70-ID)785001
=040  \\$aABB060$bcze
=080  \\$a7.01$2MRF
=080  \\$a(0:82-4)$2MRF
=100  1\$aKliment, Alexandr,$d1929-2017$7jk01060558$4aut
=245  10$aKultura je když... /$cAlexandr Kliment.
=500  \\$aPodnázev zdrojového dokumentu: (London, Index on Censorship).
=520  2\$aÚvaha.
=599  \\$aCLB-CPK
=650  07$aestetika$7ph117189$2czenas
=655  \7$aúvahy$7fd134000$2czenas
=773  0\$tSpektrum$gR. 1978, č. 1, srpen, s. 77-79$q1978:1<77$9197808
=964  \\$aB70
=964  \\$aSMZ
=964  \\$aCLE
=OWN  \\$aUCLA
=SIF  \\$afk
=910  \\$aABB060
=ZAZ  \\$d197808$s1978$z1$l77



Vidíme, že marcový soubor má jasnou strukuru. Má několik polí, která jsou označena zpravidla třemi číslicem, případně třemi písmeny. Každý kód má svou vnitřní logiku, např. pole pro věcné popisy vždy začínají číslicí ``6XX``. 
Za číslem - tagem pole se obvyke nachází dva indikátory. Pokud indikátor není definovaný, píše se místo něj zpětné lomítko (\\). 
Většina polí se dělí na podpole. Nachází se za dolarem (``$``) a označují se buď jedním písmenem, nebo číslicí.  



<div class='alert alert-block alert-info'>
    <b>Try It!</b>  Můžeme pomocí parametru ith nastavit, kolikátý záznam chceme vypsat (indexování začíná od 0). Pokud bychom chtěli vypsat všechny záznamy do i-tého záznamu, změníme `if i == ith:` na `if i <= ith:` a smažeme příkaz `break`.
</div>



Pro práci s databází pravděpodobně nebudeme potřebovat všechna pole, proto si vypíšeme jen číslo záznamu, název, autora a žánr. K většině polím se dostaneme přes tečkovou notaci (`record._`)

In [48]:
ith = 5

# Otevreni souboru
with open(database, 'rb') as data:
    # Nacteni marcu
    reader = MARCReader(data)

    # Iterace skrz nacteny soubor
    for i, record in enumerate(reader):

        if i == ith:
            # Vypsani marcovych zaznamu
            # K nekterym zaznamum muzeme pristupovat pres teckovou notaci, tedy record.leader nebo record.title 
            print("Záznam: " + record.leader)
        
            # Pred vypsanim je standardem se nejprve podivat, zda zaznam existuje (tedy ze neni None). 
            # Pokud bychom se totiz snazili vypsat None zaznam, kod by vyhodil error. 
            if record.title is not None:
                print("Nazev: " + record.title)
            if record.author is not None:
                print("Autor: " + record.author)
            # Pokud chceme zjistit, zda pole ktere nema teckovou notaci neni None, je potreba zavolat funci .get_field(), pripadne .get_field()     
            if record.get_fields('655') is not None:    
                # K polim, ktere nemaji teckovou notaci, se pristupuje pres zavorky 
                print("Žánr: " + record['655']['a'])
            break        

Záznam: 00878nab a22003134a 4500
Nazev: Archeologie času /
Autor: Vašíček, Zdeněk, 1933-2011 jk01141688 aut
Žánr: eseje


#### 1.1 Počet záznamů

Také by nás mohlo zajímat, kolik záznamů je v dané databázi. K tomu si vytvoříme samostatnou funkci, kterou pak jen zavoláme. Funkce projde celý soubor a při každém záznamu si připočítá jedničku k počítadlu `counter`.  

In [49]:
# Vlastni funkce definujeme pomoci slova def
def number_of_records(database):
    with open(database, 'rb') as data:
    # Nacteni marcu
        reader = MARCReader(data)
        # Vytvorime counter, ke kteremu pri kazdem zaznamu pricteme jednicku
        counter = 0
        # Vzhledem k tomu, ze record nepotrebujeme, tak muzeme pouzit podtrzitko (_), ktere hodnotu bude ignorovat 
        for _ in reader:
            counter += 1
    return counter 

print("V databázi je " + str(number_of_records(database)) + " záznamu.")        

V databázi je 9612 záznamu.


### 2. Uložení

S marcovým dokumentem se nepracuje příliš dobře, proto je lepší si data uložit do jednodušší tabulky. V této fázi si musíme ujasnit, jaká data budeme chtít. V našem příkladu si budeme chtít uložit název, autora, autorův kód, rok vydání a pak pole '``600 $a``','``650 $a``', '``655 $a``' a '``773 $t``'. <br>
Veškeré záznamy začínající číslem 6XX jsou věcné údaje o záznamu. Tato pole se mohou opakovat. <br>     
Pod polem '600' se skrývají osoby, o kterých záznam je nebo případně osoby, kterým je záznam dedikován. <br> 
Pod polem '650' se nacházejí věcné termíny/téma, tzn. o čem záznam je. <br>
Pod polem '655' pak najdeme žánr daného záznamu. Na rozdíl od polí '600' a '650' by pole '655' mělo být přítomné u každého záznamu. <br>
Záznamy začínající čísly 76X - 78X se nazývají propojovací pole a slouží pro zápis zdrojového (773) nebo recenzovaného (787) dokumentu. <br>


###### Pro ukládání máme připravenou funkci `save_to_dict(record, dict, field_list)`, která nám jeden záznam(record) uloží do slovníku (struktury v pythonu) dict. Není strategické si do tabulky ukládat všechna pole, podpole a indikátory, proto funkci `save_to_dict` kromě záznamu předáme také seznam polí `field_list`, která chceme uložit.  

In [50]:
def save_to_dict(record, dict, field_list):
    if not record is None:
        try:
            # Iterace skrz tuples v seznamu field_list
            for field_tags in field_list:
                # Nazev klice ve slovniku
                dict_key_name =  field_tags[0]

                # Tag pole
                tag =  field_tags[1]

                # Tag podpole
                subfield_tag =  field_tags[2]
                
                # Seznam do ktereho pridame hodnoty a nasledne pridame do slovniku
                dict_add_list = []
                
                # Iterace pres vsechna pole s tagem 'tag'
                for field in record.get_fields(tag):
                    
                    # Pokud pole nema zadna podpole, pridame cele pole do listu dict_add_list
                    if subfield_tag is None:
                        dict_add_list.append(str(field))
                    
                    # Pokud subtag je instance slice, tedy to znamena, ze chceme jen nejakou cast pole, ktera neni definovana subpolem,
                    # pridame cast pole do slovniku dict_add_list    
                    elif isinstance(subfield_tag, slice):
                        dict_add_list.append(str(field) [subfield_tag])     
                    
                    # Pokud pole obsahuje podpole, pridame do slovniku dict_add_list jen podpole
                    elif '$'+subfield_tag in str(field):  
                        dict_add_list.append(str(field[subfield_tag]))

                # Do klice z tuplu pridame cely seznam dict_add_list         
                dict[dict_key_name].append(dict_add_list)
        except Exception as error:
            print("Exception: " + type(error).__name__)  
            print("964 Field: " + str(record.get_fields('964')))  
            print("LDR: " + str(record.leader))   
    return dict 

print("Funkce uložena")

Funkce uložena


Seznam `field_list` sestává z tuplů (kolekce v jazyce Python), kde každý tuple vypadá následovně. Na první pozici je název klíče, pod kterým bude pole uloženo, na druhé pozici tag pole a na třetí tag podpole, např. ('author', '100', 'a').

In [51]:
out = 'data/csv/out_cle.csv'

with open(database, 'rb') as data:
    reader = MARCReader(data)
    # Seznam poli, ktere si chceme ulozit
    field_list = [('title', '245', 'a'),
                ('author', '100', 'a'),
                ('author code', '100', '7'),
                # Rok je schovany v poli 008 na 13. az 16. miste, proto vyuzijeme funkci slice
                ('year', '008', slice(13,17, None)),
                ('figures', '600', 'a'),
                ('description', '650', 'a'),
                ('genre', '655', 'a'),
                ('magazine', '773', 't')]
    dict = {}
    for t in field_list:
        dict_key_name = t[0]
        dict[dict_key_name] = []
    for record in reader:
        dict = save_to_dict(record, dict, field_list)
df = pd.DataFrame.from_dict(dict)

In [52]:
# Vypise poslednich 5 zaznamu v DataFramu
df.tail()

,title,author,author code,year,figures,description,genre,magazine
9607,[Co sklízet? /],"[Odložilík, Otakar,]",[jk01090958],[1953],[],"[exilový tisk, exilová literatura, časopisecké...",[úvahy],[Sklizeň (Hamburg)]
9608,"[Viktor Dyk, rytíř národní cti a básník národn...","[Beran, Jan]",[],[1957],"[Dyk, Viktor,]","[čeští spisovatelé, čeští básníci, čeští drama...","[biografické poznámky, úryvky, česká dramata, ...",[Modrá revue (Rotterdam)]
9609,[České jméno ve světě /],[],[],[1960],"[Papini, Giovanni,, Nosco, Beatrice M., Němeče...","[česká literatura, literárněvědné rozbory, rec...","[recenze, přehledy]",[Studie (Řím)]
9610,[[Television Opera Theatre...].],[],[],[1953],"[Martinů, Bohuslav,, Gogol‘, Nikolaj Vasil‘jev...","[opery, libreta, hudební skladatelé, ruští spi...",[zprávy],[Sklizeň (Hamburg)]
9611,[Být sovětským básníkem /],"[Vlach, Robert,]",[jk01150030],[1962],"[Ošanin, Lev Ivanovič,, Jesenin, Sergej Aleksa...","[ruská poezie, ruští básníci, literatura a ide...","[úvahy, ruská poezie, úryvky]",[Studie (Řím)]


#### 2.1 Úprava dat 

Vidíme, že jména za sebou mají přebytečnou čárku, kterou jednoduše odstraníme. 

In [53]:
# U jmen si chceme ulozit jmeno a prijmeni bez koncove carky ',', ktera je na konci stringu
df['figures'] = df['figures'].apply(lambda x: [y[:y.rfind(',')] if isinstance(y, str) and len(y) > 0 else y for y in x]) 
df['author'] = df['author'].apply(lambda x: [y[:y.rfind(',')] if isinstance(y, str) and len(y) > 0 else y for y in x])  

In [54]:
# Vypise poslednich 5 zaznamu v DataFramu
df.tail()

,title,author,author code,year,figures,description,genre,magazine
9607,[Co sklízet? /],"[Odložilík, Otakar]",[jk01090958],[1953],[],"[exilový tisk, exilová literatura, časopisecké...",[úvahy],[Sklizeň (Hamburg)]
9608,"[Viktor Dyk, rytíř národní cti a básník národn...",[Beran],[],[1957],"[Dyk, Viktor]","[čeští spisovatelé, čeští básníci, čeští drama...","[biografické poznámky, úryvky, česká dramata, ...",[Modrá revue (Rotterdam)]
9609,[České jméno ve světě /],[],[],[1960],"[Papini, Giovanni, Nosco, Němeček, Zdeněk, Gib...","[česká literatura, literárněvědné rozbory, rec...","[recenze, přehledy]",[Studie (Řím)]
9610,[[Television Opera Theatre...].],[],[],[1953],"[Martinů, Bohuslav, Gogol‘, Nikolaj Vasil‘jevič]","[opery, libreta, hudební skladatelé, ruští spi...",[zprávy],[Sklizeň (Hamburg)]
9611,[Být sovětským básníkem /],"[Vlach, Robert]",[jk01150030],[1962],"[Ošanin, Lev Ivanovič, Jesenin, Sergej Aleksan...","[ruská poezie, ruští básníci, literatura a ide...","[úvahy, ruská poezie, úryvky]",[Studie (Řím)]


#### 2.2 Export do csv a excelu

In [55]:
# Aby se nam list hodnot lepe ukladal, vytvorime z listu jeden string a jednotlive elementy spojime strednikem ';' 
for column in df.columns:
    df[column] = df[column].apply(lambda x: ';'.join(x))

# DataFrame ulozime do formatu csv
df.to_csv(out, encoding = 'utf8', sep = ",", index=False)   

print("Data uložena do .csv.")

# Soubor muzeme ulozit i jako excelovy soubor, se kterym dal muzeme pracovat v excelu
with pd.ExcelWriter('cle.xlsx') as writer:
    df.to_excel(writer, index=False)

print("Data uložena do .xlsx.")

Data uložena do .csv.
Data uložena do .xlsx.


### 3. Načtení z csv

Teď, když máme hodnoty uložené v csv, bude mnohem jednodušší s nimi pracovat. Uložená data načteme pomocí knihovny pandas do datové struktury DataFrame (která je podobná např. excelovské tabulce). Řádky v DataFramu reprezentují jednotlivé záznamy, sloupce pak jeden typ (např. jmeno autora).
Některá pole a podpole mohou opakovat, ty jsou  v csv spojené středníkem. V DataFramu je pak podpole rozpojíme a převede do listu (seznamu). 


###### Kvůli obecnosti jsou v listu uloženy veškeré hodnoty. Je jednodušší, když ve sloupci je pouze jeden datový typ (např. list nebo string), než když jsou některé hodnoty v listu a některé jsou pouze string. Pokud jsme si jisti, že v původních záznamech se pole nikde neopakovalo, můžeme celý sloupec převést do jiného formátu.  

In [56]:
# Cesta k nasim datum
csv_data = 'data/csv/out_cle.csv'

# Nacteni dat
df = pd.read_csv(csv_data, delimiter=',')

print("Data načtena do DataFramu df.")

for column in df.columns:
    if df[column].dtype != 'int64':   
        # Hodnoty spojene v jeden string zpatky rozdelime do listu, aby se nam s nim lepe pracovalo
        df[column] = df[column].apply(lambda x: x.split(';') if isinstance(x, str)  else [])

Data načtena do DataFramu df.


### 4. Souhrnné statistiky

Nyní se podíváme, jak z dat zjistit jednoduché statistiky. 

#### 4.1 Počet výskytů 

Pokud chceme vědet, kolikrát se v databázi objevuje nějaká hodnota, použijeme funkci `value_counts()`. V následujícím příkladu si ukážeme, kolik je v databázi z jakého roku záznamů.

In [57]:
records_to_print = 20

df['year'].value_counts()[:records_to_print] # vypise prvnich x zaznamu

year
1986    578
1987    564
1989    521
1985    519
1983    488
1988    476
1984    453
1981    423
1980    368
1982    366
1958    304
1979    298
1978    253
1959    227
1977    216
1975    212
1974    212
1976    211
1956    199
1972    187
Name: count, dtype: int64

Vidíme, že zdaleka nejvíce záznamů je z 80. let, následně pak z let 70. 

<div class='alert alert-block alert-info'>
    <b>Try It!</b> Místo sloupce 'year' můžeme vypsat např. 'author', čímž zjistíme, kolikrát se jaký autor v databázi objevuje.
</div>

#### 4.2 Nejčastější autor v daném roce

Teď si ukážeme, jak z dat zjistit, který autor v jakém roce publikoval nejvíce článků (tedy se v záznamech v daném roce nejčastěji objevuje jako autor). Existuje několik způsobů, jak na to přijít. My si představíme dva.<br>

##### 4.2.1 První varianta

Nejprve si napíšeme dvě vlastní funkce. První nám spočte počet článků každého autora v každém roce. Druhá pak pro každý rok zjistí, který autor v každém roce napsal nejvíce článků.  


In [58]:
# Funkce ktera nam spocte pocet clanku v danem roce pro kazdeho autora
def count_years(df, column):
    
    # Dictionary, ve kterem jako klic ulozime (rok, jmeno autora)
    # a jako hodnotu budeme pripocitavat pocet clanku
    count = {}
    
    # Iterace pres radky DataFramu
    for _,row in df.iterrows():
        year = row['year']
        for element in row[column]:
            key = (year, element) 
            # Pokud klic jiz existuje, pripocteme jednicku
            if key in count.keys():
                count[key] += 1
            else:
                # Pokud klic zatim neexistuje, vytvorime ho a pridame jedna
                count[key] = 1  
    return count              

# Funkce, ktera nam najde nejcastejsiho autora v danem roce
def find_most_common(count):
    # Dictionary, ve kterem jako klic ulozime rok
    # a jako hodnotu (jmeno autora, pocet clanku)
    most_common = {}

    # Iterace skrz dictionary
    for key,value in count.items():
        year = key[0]
        # Pokud klic (rok) jiz existuje, podivame se, 
        # zda je pocet clanku u aktualniho autora vyssi  
        if year in most_common:
            if most_common[year][1] < value:
                # Pamatujeme si jen tu nejvyssi hodnotu
                most_common[year] = (key[1], value)    
        else:
            most_common[year] = (key[1], value) 
    
    #Klice slovniku seradime od nejmensiho cisla po nejvetsi         
    years = list(most_common.keys())
    years.sort()
    sorted_most_common = {i: most_common[i] for i in years}        
    return sorted_most_common            

print("Funkce uloženy")
  

Funkce uloženy


Teď už jen funkce zavoláme a výsledek vypíšeme. <br> 

In [59]:
# Vybereme sloupec, ktery chceme vypsat
picked_column = 'author'

count = count_years(df, picked_column)

most_common = find_most_common(count)

# Formatovani pro hezci vypis
string = "{year: >20} {name: >20} {articles: >20}"
# Nejcastejsi hodnoty vypiseme
for key, value in most_common.items():
    print(string.format(year = key, name = value[0], articles = value[1]))        


                1948    Jandáček, Antonín                    1
                1949    Dresler, Jaroslav                    1
                1950        Demetz, Peter                    3
                1951        Demetz, Peter                    2
                1952      Peška, Vladimír                    3
                1953        Vlach, Robert                   17
                1954      Hostovský, Egon                    8
                1955        Vlach, Robert                   13
                1956       Želivan, Pavel                   11
                1957        Vlach, Robert                    9
                1958            Den, Petr                   16
                1959            Den, Petr                   16
                1960    Dresler, Jaroslav                   18
                1961            Den, Petr                   12
                1962            Den, Petr                   16
                1963            Den, Petr              

V tabulce vidíme, že počet napsaných článků za jednotlivé roky se výrazně liší. 

<div class='alert alert-block alert-info'>
    <b>Try It!</b> Kód je napsán tak, aby se parametry daly jednoduše upravit. Můžeme si tak zkusit změnit proměnnou `picked_colum` na 'figures', čímž zjistíme, o kom se nejvíce daný rok psalo.
</div>

#### 4.2.2 Druhá varianta

Tím, že máme data uložena do DataFramu, umožňuje nám to využít funkce knihovny pandas. V našem případě využijeme funkci , která data seskupí podle vybraného sloupce. <br>
Použijeme také funkci `Counter`, která nám spočte výskyt každého elementu. <br>
V neposlední řadě využijeme anonymní funkci lambda.<br>

In [60]:
# Data ocistime o prazdne hodnoty
data_filtered = df[df[picked_column].apply(lambda x: len(x) > 0)]

# Data seskupime podle nejcastejsiho elementu v kazde hodnote groupby
most_common = data_filtered.groupby('year')[picked_column].apply(lambda x: Counter(element for elements in x for element in elements).most_common(1)[0][0] if len(x) > 0 else None) 

print(most_common)

year
1948      Jandáček, Antonín
1949      Dresler, Jaroslav
1950          Demetz, Peter
1951          Demetz, Peter
1952        Peška, Vladimír
1953          Vlach, Robert
1954        Hostovský, Egon
1955          Vlach, Robert
1956         Želivan, Pavel
1957          Vlach, Robert
1958              Den, Petr
1959              Den, Petr
1960      Dresler, Jaroslav
1961              Den, Petr
1962              Den, Petr
1963              Den, Petr
1964              Den, Petr
1965              Den, Petr
1966              Den, Petr
1967              Den, Petr
1968              Den, Petr
1969              Den, Petr
1970              Den, Petr
1971    Kratochvil, Antonín
1972    Kratochvil, Antonín
1973    Kratochvil, Antonín
1974      Dresler, Jaroslav
1975    Kratochvil, Antonín
1976      Dresler, Jaroslav
1977      Dresler, Jaroslav
1978      Dresler, Jaroslav
1979      Dresler, Jaroslav
1980           Kovtun, Jiří
1981    Kratochvil, Antonín
1982    Kratochvil, Antonín
1983      Dresl

V tabulce můžeme vidět, který autor za daný rok publikoval nejvíce článků napříč všemi časopisy. Je třeba si uvědomit, že i když je v tabulce nějaký autor v daný rok nejčastější, nemusí v celkovém počtu napsat hodně článků. Ostatně se o tom můžeme přesvěsvědčit v předchozí buňce.<br>
Je možné, že se od sebe dvě tabulky budou mírně odlišovat a to v případě, že za daný rok napsalo více autorů stejný počet článků. 

<div class='alert alert-block alert-info'>
    <b>Try It!</b> Kromě bibliografie literárního exilu můžeme vyzkoušet i jiné bibliografie.
</div>
